In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTENC
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import NearMiss
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from sklearn.model_selection import StratifiedKFold
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.metrics import make_scorer, recall_score, f1_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from sklearn.metrics import roc_auc_score

In [1]:
import lightgbm as lgb
print(lgb.__version__)

# 测试LightGBM的GPU支持
params = {
    'boosting_type': 'gbrt',
    'objective': 'binary',
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0
}

# 创建一个简单的数据集并训练模型
import numpy as np

# 创建数据
X_train = np.random.rand(100, 10)
y_train = np.random.randint(2, size=100)

train_data = lgb.Dataset(X_train, label=y_train)

# 训练模型
bst = lgb.train(params, train_data, num_boost_round=10)


4.4.0
[LightGBM] [Info] Number of positive: 44, number of negative: 56


LightGBMError: GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1

# 1. Load Data and Preprocessing

- label encoding: deal with categorical features;
    - one hot encoding: 生成更多特征
- pick the features as X and the target as y;
- split the data into training and testing sets; (选择前六个月0-5作为训练集，这是作者提到的一个实践方式）
- scale the numerical features(MinMaxScaler);由于大量特征的分布并不趋于正态，有大量异常值等，选择minmax更好
- store the indices of the categorical features(for later use in SMOTE-NC);

## 1.1 Encoding Categorical Features

In [2]:
base = pd.read_csv('datasets/processed_base.csv')  # no missing values. dropped 3 cols

In [3]:
# one-hot encoding
def one_hot_encoding(data):
    # 需要进行One-Hot Encoding的分类变量
    categorical_features = ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
    
    # 使用pd.get_dummies进行One-Hot Encoding
    # drop_first=True表示删除第一个类别，避免多重共线性,同时避免引入不必要的特征并且保留了所有信息
    # 此时先用False保留所有特征，后续建立模型时候可以选择drop_first=True来避免多重共线性并减少特征数量
    data_encoded = pd.get_dummies(data, columns=categorical_features,drop_first=True)
    
    # 找出新生成的 One-Hot 编码列
    encoded_columns = data_encoded.columns.difference(data.columns)
    
    # 将 One-Hot 编码列转换为整数类型
    data_encoded[encoded_columns] = data_encoded[encoded_columns].astype(int)
    
    return data_encoded

# 示例预处理
df = one_hot_encoding(base)
df.head()

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,...,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG,source_TELEAPP,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,0,0.3,0.986506,25.0,40,0.006735,1059,13096.035018,7850.955007,6742.080561,...,1,0,0,0,0,0,0,0,0,0
1,0,0.8,0.617426,89.0,20,0.010095,1658,9223.283431,5745.251481,5941.664859,...,1,0,0,0,0,0,0,1,0,0
2,0,0.8,0.996707,14.0,40,0.012316,1095,4471.472149,5471.988958,5992.555113,...,1,0,0,0,0,0,0,0,1,0
3,0,0.6,0.475100,14.0,30,0.006991,3483,14431.993621,6755.344479,5970.336831,...,1,0,0,0,0,0,0,0,0,0
4,0,0.9,0.842307,29.0,40,5.742626,2339,7601.511579,5124.046930,5940.734212,...,1,0,0,0,0,0,0,1,0,0


In [ ]:
# labnel encoding
def label_encoding(data):
    # 需要进行Label Encoding的分类变量
    categorical_features = ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
    
    # 创建 LabelEncoder 实例
    label_encoder = LabelEncoder()
    
    # 对每个分类变量进行 Label Encoding
    for feature in categorical_features:
        data[feature] = label_encoder.fit_transform(data[feature])
    
    return data

df = label_encoding(base)

## 1.2 Train test split & Resampling
可以选择前六个月为训练集，剩余为测试集，也可以直接随机split（base没有时间bias）

- RandomUnderSampler: 随机下采样
- NearMiss: 通过保留与最近邻居类别不同的样本来下采样

Now apply the undersampling to the training set (0:1 = 90:1-->20:1) ,later we will apply smote on the resampled train set in cross validation to make it balanced.

- Original train set: Majority class: 90, Minority class: 1
- Under-sampled train set: Majority class: 20, Minority class: 1
- Over-sampled train set: Majority class: 1, Minority class: 1  BALANCED

In [41]:
# 查看month<6的数据
# df[df['month'] < 6] # 790334

# 使用前六个月作为训练集,indices 为790334
train_indices = range(790335)
test_indices = range(790335, len(df))

# 使用loc方法根据索引范围分割数据
x_train = df.loc[train_indices, df.columns != 'fraud_bool']
y_train = df.loc[train_indices, 'fraud_bool']
x_test = df.loc[test_indices, df.columns != 'fraud_bool']
y_test = df.loc[test_indices, 'fraud_bool']

In [4]:
# train test split random
X = df.drop(columns='fraud_bool')
y = df['fraud_bool']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42, shuffle=True)

In [5]:
# 下采样函数
def undersample_majority_class(x, y, ratio=10, strategy='random'):

    minority_class_size = np.sum(y == 1)
    majority_class_size = minority_class_size * ratio
    
    if strategy == 'random':
        undersample = RandomUnderSampler(sampling_strategy={0: majority_class_size, 1: minority_class_size}, random_state=42)
    elif strategy == 'nearmiss':
        undersample = NearMiss(sampling_strategy={0: majority_class_size, 1: minority_class_size}, n_jobs=-1)
    else:
        raise ValueError("Unsupported strategy. Use 'random' or 'nearmiss'.")
    
    x_resampled, y_resampled = undersample.fit_resample(x, y)
    return x_resampled, y_resampled

In [6]:
# 若直接对整个数据集进行下采样。该方法不会导致数据泄露问题，因为只是移除某些样本，但是实际情况可能不太符合。
x_nm,y_nm=undersample_majority_class(x_train,y_train,ratio=10,strategy='nearmiss')
x_train, x_test, y_train, y_test = train_test_split(x_nm, y_nm, test_size=0.2, random_state=42, shuffle=True)
y_train.value_counts(),y_test.value_counts()

(fraud_bool
 0    83524
 1     8427
 Name: count, dtype: int64,
 fraud_bool
 0    20966
 1     2022
 Name: count, dtype: int64)

## 1.4 Scaling

In [9]:
# 筛选出二进制特征
binary_features_name = [
    'email_is_free',
    'phone_home_valid',
    'phone_mobile_valid',
    'has_other_cards',
    'foreign_request',
    'keep_alive_session',
    'fraud_bool'
]

binary_features = df[binary_features_name]

# 列出所有的数值型特征
# numeric_features_name = df.select_dtypes(include=['number']).columns.tolist()
# 去掉二进制特征
# numeric_features_name = [col for col in numeric_features_name if col not in binary_features_name]

numeric_features_name=[x for x in df.columns if df[x].nunique() >= 10]

numeric_features = df[numeric_features_name]

# 创建 MinMaxScaler 实例
numeric_transformer = MinMaxScaler()

# 创建 ColumnTransformer
preprocessor = ColumnTransformer([('scaled', numeric_transformer, numeric_features_name)], remainder='passthrough')

# 在训练集上拟合并转换
x_train_scaled = preprocessor.fit_transform(x_train)

# 仅转换测试集
x_test_scaled = preprocessor.transform(x_test)

# 获取列的顺序(colmuntransformer会改变列的顺序)
new_columns = numeric_features_name + [col for col in x_train.columns if col not in numeric_features_name]

# 转换后的结果通常为 NumPy 数组，将其转换回 DataFrame 并保持列名一致性
x_train_scaled = pd.DataFrame(x_train_scaled, columns=new_columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=new_columns)

# 确定分类特征的名称
categorical_features_name = [col for col in x_train_scaled.columns if 2 <= x_train_scaled[col].nunique() < 10]

# 确定分类特征的索引
all_features = list(x_train_scaled.columns)  # 所有特征的名称列表
categorical_feature_indices = [i for i, feature in enumerate(all_features) if feature in categorical_features_name]

print("分类特征名称：", categorical_features_name)
print("分类特征索引：", categorical_feature_indices)

分类特征名称： ['income', 'customer_age', 'email_is_free', 'phone_home_valid', 'phone_mobile_valid', 'has_other_cards', 'foreign_request', 'keep_alive_session', 'device_distinct_emails_8w', 'month', 'payment_type_AB', 'payment_type_AC', 'payment_type_AD', 'payment_type_AE', 'employment_status_CB', 'employment_status_CC', 'employment_status_CD', 'employment_status_CE', 'employment_status_CF', 'employment_status_CG', 'housing_status_BB', 'housing_status_BC', 'housing_status_BD', 'housing_status_BE', 'housing_status_BF', 'housing_status_BG', 'source_TELEAPP', 'device_os_macintosh', 'device_os_other', 'device_os_windows', 'device_os_x11']
分类特征索引： [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]


In [10]:
# 对标准化后的数据下采样
x_train_nm, y_train_nm = undersample_majority_class(x_train_scaled, y_train, ratio=20, strategy='nearmiss')
y_train.value_counts(),y_train_nm.value_counts(),y_test.value_counts()

AttributeError: 'NoneType' object has no attribute 'split'

# 2. Modeling
使用未经过重采样的数据集进行建模，作为baseline model，并与之后重采样的模型进行对比。

准备使用的模型：Logistic Regression，Random Forest，XGBoost

## 2.1 Train Functions

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

def train_classifier(classifier, param_distributions, x_train_scaled, y_train, x_train_resampled, y_train_resampled, search_type='random', n_iter=50, cv=5, scoring='f1'):
    
    if search_type == 'random':
        search_cv = RandomizedSearchCV(classifier, param_distributions, n_iter=n_iter, cv=cv, scoring=scoring, n_jobs=-1, random_state=42)
    elif search_type == 'grid':
        search_cv = GridSearchCV(classifier, param_distributions, cv=cv, scoring=scoring, n_jobs=-1)
    else:
        raise ValueError("search_type should be either 'random' or 'grid'")
    
    # Train on the original (scaled) data
    search_cv.fit(x_train_scaled, y_train)
    best_model_original = search_cv.best_estimator_
    
    # Train on the resampled data
    search_cv.fit(x_train_resampled, y_train_resampled)
    best_model_resampled = search_cv.best_estimator_
    
    return best_model_original, best_model_resampled

In [99]:
def train_classifier(classifier, param_dist, X_train, y_train, encoded_features, search_type='random', scoring='roc_auc', resample_first=True, iterations=10):
    
    # 转换为numpy数组并确保数据类型高效
    X_train = np.array(X_train).astype(np.float32)
    y_train = np.array(y_train).astype(np.int32)
    
    # 创建交叉验证策略
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 选择评分方法
    if scoring == 'f1':
        scoring_method = make_scorer(f1_score, pos_label=1)  # pos_label=1 表示正类标签为1
    elif scoring == 'recall':
        scoring_method = make_scorer(recall_score, pos_label=1)
    else:
        scoring_method = scoring

    # 构建采样和模型管道
    smote_nc = SMOTENC(categorical_features=encoded_features, sampling_strategy='minority', random_state=42, n_jobs=-1)
    if resample_first:
        X_train, y_train = smote_nc.fit_resample(X_train, y_train)
        pipeline = make_pipeline(classifier)
    else:
        pipeline = make_pipeline(smote_nc, classifier)
    
    # 进行超参数搜索
    if search_type == 'random':
        search_cv = RandomizedSearchCV(estimator=pipeline, 
                                       param_distributions=param_dist,
                                       n_iter=iterations,
                                       scoring=scoring_method,
                                       n_jobs=-1,
                                       cv=cv,
                                       verbose=2)
    elif search_type == 'grid':
        search_cv = GridSearchCV(estimator=pipeline,
                                 param_grid=param_dist,
                                 scoring=scoring_method,
                                 n_jobs=-1,
                                 cv=cv,
                                 verbose=2)
    else:
        raise ValueError('search_type 必须是 "random" 或 "grid"')
        
    # 拟合模型并返回训练好的分类器
    search_cv.fit(X_train, y_train)

    return search_cv

## 2.2 Evaluate Functions

In [71]:
# print classification report
def print_cls_report(y_test, y_pred, title="Classification Report"):
    report = classification_report(y_test, y_pred, output_dict=True)
    recall = report["1"]["recall"]
    print(f"{title}:\n", classification_report(y_test, y_pred))
    return recall

# plot confusion matrix
def plot_con_matrix(ax, y_test, y_pred, title):
    # Define the classes of the classification problem
    classes = ['No Fraud', 'Fraud']

    # Compute the confusion matrix
    con_matrix = confusion_matrix(y_test, y_pred)

    # Compute the values for true negatives, false positives, false negatives, and true positives
    tn, fp, fn, tp = con_matrix.ravel()

    # Compute the false positive rate
    fpr = fp / (fp + tn)

    # Plot the confusion matrix using a heatmap
    ax.imshow(con_matrix, interpolation='nearest', cmap=plt.cm.Blues)

    # Define the tick marks and the labels for the plot
    tick_marks = np.arange(len(classes))
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(classes)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(classes)

    # Add the count of each cell of the confusion matrix to the plot
    fmt = 'd'
    threshold = con_matrix.max() / 2.
    for i, j in np.ndindex(con_matrix.shape):
        ax.text(j, i, format(con_matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if con_matrix[i, j] > threshold else "black")

    # Add labels to the plot
    ax.set_xlabel('Predicted label')
    ax.set_ylabel('True label')
    ax.set_title(f'{title} with {fpr*100:.2f}% FPR')

In [72]:
# labels = y_test
groups = (x_test["customer_age"] > 50).map({True: ">50", False: "<=50"}) 

def get_fairness_metrics(y_true, y_pred, groups, FIXED_FPR):
    g = Group()
    aequitas_df = pd.DataFrame(
        {"score": y_pred,
         "label_value": y_true,
         "group": groups}
    )
    disparities_df = g.get_crosstabs(aequitas_df, score_thresholds={"score_val": [FIXED_FPR]})[0]
    predictive_equality = disparities_df["fpr"].min() / disparities_df["fpr"].max()
    return predictive_equality, disparities_df

def evaluate_classifier(classifier, x_train,y_train,x_test, y_test, groups):
    y_train_pred = classifier.predict(x_train)
    y_pred = classifier.predict(x_test)
    y_prob = classifier.predict_proba(x_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, y_prob)

    target_fpr = 0.05  # Set the target threshold at 5% FPR
    threshold_idx = np.argmin(np.abs(fpr - target_fpr))
    threshold = thresholds[threshold_idx]
    
    y_pred_threshold = (y_prob >= threshold).astype(int)
    
    train_set = print_cls_report(y_train, y_train_pred, title="Train Set")
    default_recall = print_cls_report(y_test, y_pred, title="Default Threshold")
    target_recall = print_cls_report(y_test, y_pred_threshold, title=f'Target Threshold @ {threshold:.2f}')
    
     # Calculate predictive equality
    predictive_equality, disparities_df = get_fairness_metrics(y_test, y_pred_threshold, groups, target_fpr)
    print(f"Predictive Equality: {predictive_equality:.2f}")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    
    default_matrix = plot_con_matrix(ax1, y_test, y_pred, title='Default Threshold @ 0.50')
    target_matrix = plot_con_matrix(ax2, y_test, y_pred_threshold, title=f'Target Threshold @ {threshold:.2f}')

    plt.tight_layout()
    plt.show()

    return fpr, tpr, default_recall, target_recall, predictive_equality

## 2.1 Random Forest

In [90]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# Define a dictionary of hyperparameters for RandomForestClassifier
rf_params = {'randomforestclassifier__n_estimators': [20, 40, 60, 80, 100],
            'randomforestclassifier__criterion': ['gini', 'entropy'],
            'randomforestclassifier__max_depth': [2, 4, 6, 8, 10],
            'randomforestclassifier__max_features': ['sqrt', 'log2']
            }

rf_model = train_classifier(rf, rf_params, x_train_scaled, y_train, categorical_feature_indices, search_type='random', scoring='recall')

Fitting 5 folds for each of 20 candidates, totalling 100 fits


ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "E:\Programming\anaconda3\envs\mytorch\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "E:\Programming\anaconda3\envs\mytorch\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "E:\Programming\anaconda3\envs\mytorch\lib\site-packages\sklearn\pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "E:\Programming\anaconda3\envs\mytorch\lib\site-packages\sklearn\pipeline.py", line 350, in _fit
    self._validate_steps()
  File "E:\Programming\anacon